In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 500
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 500
N_FEATURES = 10

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 160, 16

In [12]:
# def step_decay(epoch):
#    initial_lrate = 0.1
#    drop = 0.1
#    epochs_drop = 16.0
#    lrate = initial_lrate * tf.math.pow(drop,  
#            tf.math.floor((1+epoch)/epochs_drop))
#    return lrate

In [13]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 70.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [14]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [18]:
# lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [19]:
# import os
path='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/'

In [20]:
for i in range(1,28):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
       
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    print('y_train shape',y_train.shape)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
#     print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 25, 20
    n_depth=10
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
# naveen architecture
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=9, padding='same', kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04)), \
                           input_shape=(n_steps,n_length,n_depth)))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=8,strides=2)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Dropout(0.2093)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Dropout(0.2093)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Dropout(0.2093)))
    model.add(TimeDistributed(Activation('relu')))
    model.add(TimeDistributed(Flatten()))
    # model.add(Flatten())
    model.add(Bidirectional(LSTM(200,return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(200,return_sequences=True)))
    model.add(Dropout(0.3))
    # model.add(Flatten())
    # model.add(Bidirectional(LSTM(200,return_sequences=True)))
    # model.add(Dropout(0.3))
    # model.add(Bidirectional(LSTM(200,return_sequences=True)))
    # model.add(Dropout(0.3))
    # model.add(Flatten())
    # model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
    # model.add(Bidirectional(LSTM(200)))
    # model.add(Dropout(0.3))
    model.add(Flatten())
    # model.add(Dense(2000, activation='tanh'))
    model.add(Dense(512, activation='tanh'))
    # model.add(Dense(200, activation='tanh'))
    model.add(BatchNormalization(epsilon=1e-05, momentum=0.9, weights=None))
    model.add(Dense(n_outputs, activation='softmax'))
    #     sgd=optimizers.SGD(learning_rate=2.17e-3, momentum=0.9, nesterov=False, name='SGD')
    #     model = Sequential()
#     model.add(TimeDistributed(Conv1D(filters=64, kernel_size=9, padding='same', kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04)), \
    #                            input_shape=(n_steps,n_length,n_depth)))
#     # model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
#     model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
#     model.add(TimeDistributed(Activation('tanh')))
#     model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
#     # model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
#     model.add(TimeDistributed(Activation('tanh')))
#     model.add(TimeDistributed(Dropout(0.2093)))
#     model.add(TimeDistributed(Activation('relu')))
#     model.add(Flatten())
#     model.add(Dense(512, activation='tanh'))
#     model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
#     model.add(Dense(n_outputs, activation='softmax'))
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999,epsilon=1e-08, amsgrad=False)
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    print('test_mean for %d subjects:',i)
    test_mean=statistics.mean(test_acc)
    print(test_mean)

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 182000

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 78000

x_train shape:  (363, 500, 10)
363 training samples
y_train shape:  (363,)
num_time_periods 500
num_sensors 10
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (500, 10)
input_shape: (500, 10)
y_train shape (363,)
New y_train shape:  (363, 52)
x_test shape:  (155, 500, 10)
155 testing samples
y_test shape:  (155,)
x_train shape:  (363, 25, 20, 10)
x_test shape:  (155, 25, 20, 1

Epoch 36/160
22/23 [===========================>..] - ETA: 0s - loss: 0.2280 - accuracy: 0.9830
Epoch 00036: val_accuracy improved from 0.90323 to 0.90968, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
23/23 [==============================] - 1s 28ms/step - loss: 0.2267 - accuracy: 0.9835 - val_loss: 0.5832 - val_accuracy: 0.9097 - lr: 0.0010
Epoch 37/160
22/23 [===========================>..] - ETA: 0s - loss: 0.2105 - accuracy: 0.9943
Epoch 00037: val_accuracy did not improve from 0.90968
23/23 [==============================] - 1s 25ms/step - loss: 0.2210 - accuracy: 0.9917 - val_loss: 0.5467 - val_accuracy: 0.8839 - lr: 0.0010
Epoch 38/160
22/23 [===========================>..] - ETA: 0s - loss: 0.2342 - accuracy: 0.9773
Epoch 00038: val_accuracy did not improve from 0.90968
23/23 [==============================] - 1s 25ms/step - loss: 0.2349 - accuracy: 0.9752 - val_loss: 0.5882 - val_accuracy: 0.8839 - lr: 0.0010
Epoch 39/160
22/2

Epoch 63/160
22/23 [===========================>..] - ETA: 0s - loss: 0.2004 - accuracy: 0.9886
Epoch 00063: val_accuracy did not improve from 0.92903
23/23 [==============================] - 1s 23ms/step - loss: 0.1999 - accuracy: 0.9890 - val_loss: 0.5972 - val_accuracy: 0.9161 - lr: 0.0010
Epoch 64/160
22/23 [===========================>..] - ETA: 0s - loss: 0.2008 - accuracy: 0.9858
Epoch 00064: val_accuracy did not improve from 0.92903
23/23 [==============================] - 1s 24ms/step - loss: 0.2004 - accuracy: 0.9862 - val_loss: 0.6087 - val_accuracy: 0.9097 - lr: 0.0010
Epoch 65/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1744 - accuracy: 1.0000
Epoch 00065: val_accuracy did not improve from 0.92903
23/23 [==============================] - 1s 24ms/step - loss: 0.1741 - accuracy: 1.0000 - val_loss: 0.5557 - val_accuracy: 0.8968 - lr: 0.0010
Epoch 66/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1984 - accuracy: 0.9858
Epoch 00066: val_accur

Epoch 91/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1626 - accuracy: 1.0000
Epoch 00091: val_accuracy did not improve from 0.92903
23/23 [==============================] - 1s 25ms/step - loss: 0.1634 - accuracy: 1.0000 - val_loss: 0.5693 - val_accuracy: 0.9097 - lr: 1.0000e-04
Epoch 92/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1632 - accuracy: 1.0000
Epoch 00092: val_accuracy did not improve from 0.92903
23/23 [==============================] - 1s 24ms/step - loss: 0.1631 - accuracy: 1.0000 - val_loss: 0.5842 - val_accuracy: 0.9032 - lr: 1.0000e-04
Epoch 93/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1626 - accuracy: 1.0000
Epoch 00093: val_accuracy did not improve from 0.92903
23/23 [==============================] - 1s 25ms/step - loss: 0.1624 - accuracy: 1.0000 - val_loss: 0.5711 - val_accuracy: 0.9097 - lr: 1.0000e-04
Epoch 94/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1631 - accuracy: 1.0000
Epoch 0009

Epoch 119/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1580 - accuracy: 1.0000
Epoch 00119: val_accuracy did not improve from 0.92903
23/23 [==============================] - 1s 26ms/step - loss: 0.1579 - accuracy: 1.0000 - val_loss: 0.5555 - val_accuracy: 0.9226 - lr: 1.0000e-04
Epoch 120/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1560 - accuracy: 1.0000
Epoch 00120: val_accuracy did not improve from 0.92903
23/23 [==============================] - 1s 25ms/step - loss: 0.1560 - accuracy: 1.0000 - val_loss: 0.5588 - val_accuracy: 0.9161 - lr: 1.0000e-04
Epoch 121/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1571 - accuracy: 1.0000
Epoch 00121: val_accuracy did not improve from 0.92903
23/23 [==============================] - 1s 25ms/step - loss: 0.1571 - accuracy: 1.0000 - val_loss: 0.5616 - val_accuracy: 0.9226 - lr: 1.0000e-04
Epoch 122/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1550 - accuracy: 1.0000
Epoch 

...

23/23 [==============================] - 1s 29ms/step - loss: 0.5669 - accuracy: 0.8705 - val_loss: 0.6263 - val_accuracy: 0.8839 - lr: 0.0010
Epoch 11/160
22/23 [===========================>..] - ETA: 0s - loss: 0.3985 - accuracy: 0.9460
Epoch 00011: val_accuracy improved from 0.88387 to 0.90323, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
23/23 [==============================] - 1s 31ms/step - loss: 0.3947 - accuracy: 0.9477 - val_loss: 0.5643 - val_accuracy: 0.9032 - lr: 0.0010
Epoch 12/160
22/23 [===========================>..] - ETA: 0s - loss: 0.3685 - accuracy: 0.9432
Epoch 00012: val_accuracy did not improve from 0.90323
23/23 [==============================] - 1s 29ms/step - loss: 0.3748 - accuracy: 0.9394 - val_loss: 0.5632 - val_accuracy: 0.8903 - lr: 0.0010
Epoch 13/160
23/23 [==============================] - ETA: 0s - loss: 0.4273 - accuracy: 0.9256
Epoch 00013: val_accuracy did not improve from 0.90323
23/23 [==========

Epoch 37/160
22/23 [===========================>..] - ETA: 0s - loss: 0.2440 - accuracy: 0.9801
Epoch 00037: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 27ms/step - loss: 0.2422 - accuracy: 0.9807 - val_loss: 0.3803 - val_accuracy: 0.9355 - lr: 0.0010
Epoch 38/160
22/23 [===========================>..] - ETA: 0s - loss: 0.2117 - accuracy: 0.9915
Epoch 00038: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 26ms/step - loss: 0.2131 - accuracy: 0.9917 - val_loss: 0.4468 - val_accuracy: 0.9032 - lr: 0.0010
Epoch 39/160
22/23 [===========================>..] - ETA: 0s - loss: 0.2085 - accuracy: 0.9972
Epoch 00039: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 26ms/step - loss: 0.2088 - accuracy: 0.9972 - val_loss: 0.4666 - val_accuracy: 0.9161 - lr: 0.0010
Epoch 40/160
22/23 [===========================>..] - ETA: 0s - loss: 0.2135 - accuracy: 0.9915
Epoch 00040: val_accur

Epoch 65/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1726 - accuracy: 1.0000
Epoch 00065: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 25ms/step - loss: 0.1724 - accuracy: 1.0000 - val_loss: 0.4321 - val_accuracy: 0.9355 - lr: 0.0010
Epoch 66/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1716 - accuracy: 0.9972
Epoch 00066: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 23ms/step - loss: 0.1716 - accuracy: 0.9972 - val_loss: 0.4393 - val_accuracy: 0.9355 - lr: 0.0010
Epoch 67/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1876 - accuracy: 0.9943
Epoch 00067: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 24ms/step - loss: 0.1871 - accuracy: 0.9945 - val_loss: 0.4290 - val_accuracy: 0.9161 - lr: 0.0010
Epoch 68/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1732 - accuracy: 0.9972
Epoch 00068: val_accur

Epoch 93/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1628 - accuracy: 1.0000
Epoch 00093: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 24ms/step - loss: 0.1627 - accuracy: 1.0000 - val_loss: 0.3663 - val_accuracy: 0.9419 - lr: 1.0000e-04
Epoch 94/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1626 - accuracy: 1.0000
Epoch 00094: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 24ms/step - loss: 0.1625 - accuracy: 1.0000 - val_loss: 0.3719 - val_accuracy: 0.9419 - lr: 1.0000e-04
Epoch 95/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1638 - accuracy: 1.0000
Epoch 00095: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 25ms/step - loss: 0.1646 - accuracy: 1.0000 - val_loss: 0.3604 - val_accuracy: 0.9419 - lr: 1.0000e-04
Epoch 96/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1644 - accuracy: 1.0000
Epoch 0009

Epoch 121/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1612 - accuracy: 1.0000
Epoch 00121: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 26ms/step - loss: 0.1619 - accuracy: 1.0000 - val_loss: 0.3592 - val_accuracy: 0.9419 - lr: 1.0000e-04
Epoch 122/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1602 - accuracy: 1.0000
Epoch 00122: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 25ms/step - loss: 0.1602 - accuracy: 1.0000 - val_loss: 0.3450 - val_accuracy: 0.9419 - lr: 1.0000e-04
Epoch 123/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1584 - accuracy: 1.0000
Epoch 00123: val_accuracy did not improve from 0.95484
23/23 [==============================] - 1s 25ms/step - loss: 0.1584 - accuracy: 1.0000 - val_loss: 0.3531 - val_accuracy: 0.9419 - lr: 1.0000e-04
Epoch 124/160
22/23 [===========================>..] - ETA: 0s - loss: 0.1581 - accuracy: 1.0000
Epoch 

In [21]:
statistics.mean(test_acc)

0.9577060915805675